In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, hparams):
        super(CNN, self).__init__()
        self.hparams = hparams
        num_conv_layers = hparams['num_conv_layers']
        in_channels = 3
        num_filters = hparams['num_filters']
        kernel_size = hparams['kernel_size']  # Single integer value
        num_classes = 10
        num_neurons_dense = hparams['num_neurons_dense']
        input_size = 224
        filter_organization = hparams['filter_organization']
        batch_normalization = hparams['batch_normalization']
        dropout_prob = hparams['dropout_prob']
        conv_activation = hparams['conv_activation']

        self.conv_layers = nn.ModuleList()
        self.num_conv_layers = num_conv_layers

        # Add convolution layers
        for i in range(num_conv_layers):
            # Determine the number of filters for this layer based on filter_organization
            if filter_organization == 'same':
                out_channels = num_filters
            elif filter_organization == 'double':
                out_channels = num_filters * (2 ** i)
            elif filter_organization == 'halve':
                out_channels = num_filters // (2 ** i)
            else:
                raise ValueError("Invalid filter organization")

            # Determine padding value to maintain spatial dimensions
            padding = kernel_size // 2 if kernel_size % 2 == 1 else (kernel_size - 1) // 2

            # Add convolution layer with the same kernel size for all layers
            conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
            self.conv_layers.append(conv_layer)

            # Add batch normalization if enabled
            if batch_normalization:
                bn_layer = nn.BatchNorm2d(out_channels)
                self.conv_layers.append(bn_layer)

            # Add dropout layer
            dropout_layer = nn.Dropout2d(p=dropout_prob)
            self.conv_layers.append(dropout_layer)

            in_channels = out_channels

        # Calculate input size for dense layer
        dense_input_size = out_channels * (input_size // (2 ** num_conv_layers)) ** 2

        # Dense layer
        self.dense = nn.Linear(dense_input_size, num_neurons_dense)

        # Dropout layer after dense layer
        self.dropout_dense = nn.Dropout(p=dropout_prob)

        # Output layer
        self.output = nn.Linear(num_neurons_dense, num_classes)

    def forward(self, x):
        # Convolution layers
        for layer in self.conv_layers:
            # Apply the appropriate activation function
            if isinstance(layer, nn.Conv2d):
                if self.hparams['conv_activation'] == 'relu':
                    x = F.relu(layer(x))
                elif self.hparams['conv_activation'] == 'gelu':
                    x = F.gelu(layer(x))
                elif self.hparams['conv_activation'] == 'silu':
                    x = F.silu(layer(x))
                elif self.hparams['conv_activation'] == 'mish':
                    x = F.mish(layer(x))
                else:
                    raise ValueError("Invalid convolutional activation function specified")
            else:
                x = layer(x)
            if isinstance(layer, nn.Conv2d):
                x = F.max_pool2d(x, 2)

        # Flatten
        x = torch.flatten(x, 1)

        # Dense layer with dropout
        x = F.relu(self.dense(x))
        x = self.dropout_dense(x)

        # Output layer (raw scores)
        x = self.output(x)
        return x

# Example usage:
hparams = {
    'num_conv_layers': 5,
    'in_channels': 0,
    'num_filters': 32,
    'kernel_size': 5,
    'num_classes': 0,
    'num_neurons_dense': 100,
    'input_size': 0,
    'filter_organization': 'same',
    'data_augmentation': False,
    'batch_normalization': False,  # Set to True or False as needed
    'dropout_prob': 0.2,
    'conv_activation': 'mish'
}
model = CNN(hparams)

print(model)

CNN(
  (conv_layers): ModuleList(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Dropout2d(p=0.2, inplace=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): Dropout2d(p=0.2, inplace=False)
    (4): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): Dropout2d(p=0.2, inplace=False)
    (6): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): Dropout2d(p=0.2, inplace=False)
    (8): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): Dropout2d(p=0.2, inplace=False)
  )
  (dense): Linear(in_features=1568, out_features=100, bias=True)
  (dropout_dense): Dropout(p=0.2, inplace=False)
  (output): Linear(in_features=100, out_features=10, bias=True)
)
